# Разбор домашки 😲

## Задача A. Оставить различные

Дан массив a из n целых чисел. Напишите программу, которая выведет:
1. исходный список чисел без дубликатов с сохранением порядка;
2. число чисел, которые были удалены из массива.

In [1]:
from random import randrange

a = [randrange(10) for _ in range(1000)]

In [2]:
def unique_ordered(a):
    visited = set()
    for e in a:
        if e in visited:
            continue
        visited.add(e)
        yield e

In [3]:
%%timeit

list(unique_ordered(a))

29.1 µs ± 435 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [4]:
def unique_ordered(a):
    visited = set()
    yield from (visited.add(e) or e for e in a if e not in visited)

In [5]:
%%timeit

list(unique_ordered(a))

27.7 µs ± 494 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [6]:
def unique_ordered(a):
    visited = set()
    return [visited.add(e) or e for e in a if e not in visited]

In [7]:
%%timeit

unique_ordered(a)

27.4 µs ± 713 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [8]:
a = [5, 5, 5, 4, 4]
b = unique_ordered(a)

print(*b)
print(len(a) - len(b))

5 4
3


## Задача B. Сложная сортировка

Дан массив a1,...,an из n натуральных чисел. Требуется отсортировать числа в массиве в порядке возрастания суммы цифр, а при равной сумме цифр — по возрастанию самого числа.

In [9]:
from random import randrange

a_int = [randrange(1000) for _ in range(1000)]
a_str = list(map(str, a_int))

In [10]:
def sum_digits(n):
    return sum(map(int, n))   # строки – итерируемые объекты

sum_digits('123')

6

In [11]:
%%timeit

list(map(sum_digits, a_str))

591 µs ± 16 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [12]:
def sum_digits(n):
    s = 0
    while n:
        s += n % 10
        n //= 10
    return s

sum_digits(123)

6

In [13]:
%%timeit

list(map(sum_digits, a_int))

258 µs ± 6.93 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [14]:
s_str = ['14', '23', '22']
s_int = list(map(int, s_str))

In [15]:
sorted(s_int, key=lambda n: (sum_digits(n), int(n)))

[22, 14, 23]

## Задача D.  Переупорядочивание списка

Имеется функция `print_by_cols(v, cols)`, которая печатает список `v` в виде таблицы из `cols` колонок. Таблица “заполняется” построчно.

```python
def print_by_cols(v, cols):
    for i in range(0, len(v), cols):
        print(*v[i:i+cols], sep='\t')
```

Напишите, функцию `reorder_list(v, cols)`, которая бы переупорядочила список `v` так, чтобы `print_by_cols` напечатала таблицу, в которой значения “заполнялись” бы поколоночно.

In [16]:
def print_by_cols(v, cols):
    for i in range(0, len(v), cols):
        print(*v[i:i+cols], sep='\t')

v = list(range(10))
print_by_cols(v, 4)

0	1	2	3
4	5	6	7
8	9


In [17]:
from itertools import chain, cycle, zip_longest

def reorder_list(v, step):
    # формируем пул для каждый из столбцов
    v_pulls = [[] for _ in range(step)]
    
    # определяем длину каждого пула (квоту)
    v_pulls_len = [len(v) // step] * step
    for j in range(len(v) % step):
        v_pulls_len[j] += 1

    # создаем итератор пар (пулл, длина пула)
    v_pulls_it = cycle(zip(v_pulls, v_pulls_len))
    # получаем первую пару
    v_sub, v_len = next(v_pulls_it)

    # итерируемся по всем элементам списка
    for p in v:
        # ищем тот пул, в котором еще не закончилась квота
        while len(v_sub) >= v_len:
            v_sub, v_len = next(v_pulls_it)
        # добавляем элемент
        v_sub.append(p)
        
    # print(v_pulls)
    
    # Вспоминаем:
    # 1. как выглядит v_pulls;
    # 2. чем zip отличается от zip_longest;
    # 3. как работает распаковка для zip.
    v_pulls = chain.from_iterable(zip_longest(*v_pulls))
    v_pulls = list(filter(lambda e: e is not None, v_pulls))

    return v_pulls

In [18]:
v_ordered = reorder_list(v, 4)

print_by_cols(v_ordered, 4)

0	3	6	8
1	4	7	9
2	5


In [19]:
v_ordered

[0, 3, 6, 8, 1, 4, 7, 9, 2, 5]